In [ ]:
import ROOT

# Open the ROOT file containing the TTree with dielectron invariant mass data
file = ROOT.TFile("data_Run2022C_passHltDoubleEle33CaloIdLMWUnsLeg.root")

# Get the TTree containing the invariant mass data
bN = "bin00_el_sc_eta_m2p50Tom1p57_el_pt_10p00To40p00"
hPass = file.Get("%s_Pass"%bN)
hFail = file.Get("%s_Fail"%bN)

bNZ = 'bin06_el_sc_eta_m2p50Tom1p57_el_pt_40p00To100p00'
hZPass = file.Get("%s_Pass"%bNZ)
hZFail = file.Get("%s_Fail"%bNZ)

# Draw the histograms
c1 = ROOT.TCanvas()
c1.Divide(2,2)
c1.cd(1); hPass.Draw()
c1.cd(2); hFail.Draw()
c1.cd(3); hZPass.Draw()
c1.cd(4); hZFail.Draw()
c1.Draw()

In [ ]:
# A very nice reference: AN2010_349_v1

#Defines a probability density function which has exponential decay 
#distribution at high mass beyond the pole position (say, Z peak)  
#but turns over (i.e., error function) at low mass due to threshold 
#effect. We use this to model the background shape in Z->ll invariant mass.
def RooCMSShape(x, alpha, beta, gamma, peak):
    erf = ROOT.TMath.Erfc((alpha - x) * beta)
    u = (x - peak)*gamma
    if(u < -70):
        u = 1e20
    elif( u>70 ):
        u = 0
    else: 
        u = math.exp(-u)
    return erf*u;
    
def getWorkspace(hPass, hFail):
    # help(ROOT.RooRealVar())
    # [value] | [minValue, maxValue] | [value, minValue, maxValue]
    vars = [
        "meanP[-0.0,-5.0,5.0]","sigmaP[0.9,0.5,5.0]",
        "meanF[-0.0,-5.0,5.0]","sigmaF[0.9,0.5,5.0]",
        "acmsP[60.,50.,80.]","betaP[0.05,0.01,0.08]","gammaP[0.1, -2, 2]","peakP[90.0]",
        "acmsF[60.,50.,80.]","betaF[0.05,0.01,0.08]","gammaF[0.1, -2, 2]","peakF[90.0]",
        ]
    nBins = 1000;
    work = ROOT.RooWorkspace("w")
    work.factory("x[50,130]")
    work.var("x").setBins(nBins, "cache")
    for v in vars: 
        work.factory(v)
    rooPass = ROOT.RooDataHist("hPass", "hPass", work.var("x"), hPass)
    rooFail = ROOT.RooDataHist("hFail", "hFail", work.var("x"), hFail)
    work.Import(rooPass)
    work.Import(rooFail)
    #print(rooPass.to_numpy())

    rooPassZ = ROOT.RooDataHist("hGenZPass","hGenZPass", work.var("x"), hZPass);
    rooFailZ = ROOT.RooDataHist("hGenZFail","hGenZFail", work.var("x"), hZFail);
    work.Import(rooPassZ)
    work.Import(rooFailZ)

    nTotP = hPass.Integral()
    nTotF = hFail.Integral()
    print(nTotP, nTotF)

#    my_custom_pdf = ROOT.RooFit.bindPdf("RooCMSShape", RooCMSShape, 
#                                        ROOT.RooArgList(work.var("x"), 
#                                        work.var("acmsP"), 
#                                        work.var("betaP"), 
#                                        work.var("gammaP"),
#                                        work.var("peakP")))

    my_custom_pdf_P = ROOT.RooGenericPdf("bkgPass", "bkgPass",
                                   "exp(-0.5 * x)",
                                   ROOT.RooArgList(work.var("x")))
    my_custom_pdf_F = ROOT.RooGenericPdf("bkgFail", "bkgFail",
                                   "exp(-0.5 * x)",
                                   ROOT.RooArgList(work.var("x")))
    getattr(work, 'import')(my_custom_pdf_P)
    getattr(work, 'import')(my_custom_pdf_F)
    # help(ROOT.RooAbsPdf)
    # (name, title, minVal, maxVal)
    pdfs = [
    "Gaussian::sigResPass(x,meanP,sigmaP)",
    "Gaussian::sigResFail(x,meanF,sigmaF)",
    #"Gaussian::bkgPass(x,meanP,sigmaP)",
    #"Gaussian::bkgFail(x,meanF,sigmaF)",
    #"RooCMSShape::bkgPass(x, acmsP, betaP, gammaP, peakP)",
    #"RooCMSShape::bkgFail(x, acmsF, betaF, gammaF, peakF)",
    ]
    for p in pdfs: 
        work.factory(p)
    #Section 5 of AN2010_349_v1
    work.factory("HistPdf::sigPhysPass(x,hGenZPass,3)");
    work.factory("HistPdf::sigPhysFail(x,hGenZFail,3)");
    work.factory("FCONV::sigPass(x, sigPhysPass, sigResPass)");
    work.factory("FCONV::sigFail(x, sigPhysFail, sigResFail)");
    work.factory("nSigP[{0},{1},{2}]".format(nTotP * 0.9, nTotP * 0.5, nTotP * 1.5))
    work.factory("nBkgP[{0},{1},{2}]".format(nTotP * 0.1, nTotP * 0.5, nTotP * 1.5))
    work.factory("nSigF[{0},{1},{2}]".format(nTotF * 0.9, nTotF * 0.5, nTotF * 1.5))
    work.factory("nBkgF[{0},{1},{2}]".format(nTotF * 0.1, nTotF * 0.5, nTotF * 1.5))
    work.factory("SUM::pdfPass(nSigP*sigPass,nBkgP*bkgPass)")
    work.factory("SUM::pdfFail(nSigF*sigFail,nBkgF*bkgFail)")
    work.Print()
    return work

ws = getWorkspace(hPass, hFail)

In [ ]:
def fitHist(ws, hPass, hFail):
    pdfPass = ws.pdf("pdfPass")
    pdfFail = ws.pdf("pdfFail")

    #--------------------------------
    # Fit options
    #--------------------------------
    fitSave = ROOT.RooFit.Save()
    fitMinimizer = ROOT.RooFit.Minimizer("Minuit2", "MIGRAD")
    useMions = ROOT.RooFit.Minos(True)
    fitStrategy = ROOT.RooFit.Strategy(2)
    fitError = ROOT.RooFit.SumW2Error(False)
    
    res_pass = pdfPass.fitTo(ws.data("hFail"), fitSave, fitMinimizer, useMions, fitStrategy, fitError)
    res_fail = pdfFail.fitTo(ws.data("hPass"), fitSave, fitMinimizer, useMions, fitStrategy, fitError)

    return res_pass, res_fail

res_pass, res_fail = fitHist(ws, hPass, hFail)

In [ ]:
def getParam(ws, resP, resF):
    eff = -1
    e_eff = 0
    nSigP = ws.var("nSigP")
    nSigF = ws.var("nSigF")
    nP, e_nP = nSigP.getVal(), nSigP.getError()
    nF, e_nF = nSigF.getVal(), nSigF.getError()
    nTot = nP + nF
    eff = nP / (nP + nF)
    e_eff = 1. / (nTot * nTot) * ROOT.TMath.Sqrt(nP * nP * e_nF * e_nF + nF * nF * e_nP * e_nP)
    text = ROOT.TPaveText(0, 0.3, 1, 0.8)
    text.SetFillColor(0); text.SetBorderSize(0); text.SetTextSize(0.06)
    text.AddText("* fit status pass: {}, fail : {}".format(resP.status(), resF.status()))
    text.AddText("* eff = {:1.4f} ± {:1.4f}".format(eff, e_eff))
    def add_text_from_list(par_list):
        for ip in range(par_list.getSize()):
            vName = par_list[ip].GetName()
            text.AddText("   {} \t= {:1.3f} ± {:1.3f}".format(
                vName,
                ws.var(vName).getVal(),
                ws.var(vName).getError()))
    add_text_from_list(resP.floatParsFinal())
    add_text_from_list(resF.floatParsFinal())
    return text

def plotRes(ws, res_pass, res_fail):
    #--------------------------------
    # Plot the result
    #--------------------------------
    canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
    canvas.Divide(3, 1)

    canvas.cd(1)
    text = getParam(ws, res_pass, res_fail)
    text.Draw()
    canvas.Draw()
    
    canvas.cd(2)
    pass_frame = ws.var("x").frame()
    pass_frame.SetTitle("passing probe")
    ws.data("hPass").plotOn(pass_frame)
    ws.pdf("pdfPass").plotOn(pass_frame)
    ws.pdf("bkgPass").plotOn(pass_frame, ROOT.RooFit.LineColor(ROOT.kRed))
    pass_frame.Draw()
    canvas.Draw()

    canvas.cd(3)
    fail_frame = ws.var("x").frame()
    fail_frame.SetTitle("failing probe")
    ws.data("hFail").plotOn(fail_frame)
    ws.pdf("pdfFail").plotOn(fail_frame)
    ws.pdf("bkgFail").plotOn(fail_frame, ROOT.RooFit.LineColor(ROOT.kRed))
    fail_frame.Draw()
    canvas.Draw()
    canvas.SaveAs("tnpFit.pdf")
plotRes(ws, res_pass, res_fail)

In [ ]:
import ROOT

# Define a Python function for the Gaussian PDF
def my_gaussian_pdf(x, mean, sigma):
    arg = (x - mean) / sigma
    return ROOT.TMath.Exp(-0.5 * arg * arg) / (ROOT.TMath.Sqrt(2 * ROOT.TMath.Pi()) * sigma)

# Create variables
x = ROOT.RooRealVar("x", "Observable", -10, 10)
mean = ROOT.RooRealVar("mean", "Mean of Gaussian", 0, -10, 10)
sigma = ROOT.RooRealVar("sigma", "Width of Gaussian", 1, 0.1, 10)

# Create a RooFit function representing the Python function
my_custom_pdf = ROOT.RooFit.bindPdf("ss", my_gaussian_pdf, x, mean, sigma)

# Create a workspace
w = ROOT.RooWorkspace("w")
getattr(w, 'import')(my_custom_pdf)

# Generate toy data
data = my_custom_pdf.generate(ROOT.RooArgSet(x), 1000)

# Fit the data using the custom PDF
my_custom_pdf.fitTo(data)

# Plot the fit result
can = ROOT.TCanvas()
frame = x.frame()
data.plotOn(frame)
my_custom_pdf.plotOn(frame)
frame.Draw()
can.Draw()
can.SaveAs("check.pdf")
